In [1]:
import numpy as np
import numba
import matplotlib.pyplot as plt
from scipy.spatial.distance import hamming
from time import time

#Import the helper functions and the samplers
from BasicFunctions import *
from TabuSampler import tabu_sampler
from ZanellaSampler import zanella_sampler
from RandomWalkSampler import rw_sampler
from PointwiseSampler import pointwise_sampler 
from GelmanRubinDiagnostic import *


#Autocorrelation function as implemented by Power and Goldman
@numba.njit()
def autocorr(x, lags):
    mean=np.mean(x)
    var=np.var(x)
    xp=x-mean
    corr=[1. if l==0 else np.sum(xp[l:]*xp[:-l])/len(x)/var for l in lags]
    return np.array(corr)

In [2]:
#Generating the database
from DatabaseClass import create_databases

#Global parameters used
lmbd = 50
l = 15 #The number of categories for each record
p_cat = np.array([0.05, 0.15, 0.4, 0.2, 0.2]) #The pval vector for each category
beta = 0.3

p_match, x, y, M_truth, M_reverse_truth = create_databases(lmbd, l, p_cat, beta)

#Other parameters that are needed
N_1 = len(x)
N_2 = len(y)
num_gens = N_1*N_2


print("The golden truth p_match: ", p_match)
print("The number of generators: ", num_gens)
print("The first record of the x database:")
print(x[0, :])
print("The first record of the y database:")
print(y[0,:])
print("The golden truth M vector: ")
print(M_truth)

The golden truth p_match:  0.9498269391233346
The number of generators:  1980
The first record of the x database:
[4. 2. 1. 3. 1. 3. 4. 2. 2. 2. 2. 3. 4. 1. 1.]
The first record of the y database:
[3. 3. 4. 2. 2. 2. 2. 1. 4. 4. 4. 4. 4. 3. 2.]
The golden truth M vector: 
[21. 44.  1. 42. 12. 22. 35.  0. 43. 11.  4. 19. 33. 28. 14. 38.  2. 27.
 34. 24. 16. 40. 36.  9. 31. 10.  7. 18. 17. 25. 20. 37. 26. 15. 29. 13.
  5. 32.  8. 23.  3.  0.  6. 41. 30.]


In [3]:
#Defining the Barker balancing function
g = lambda t: t/(1+t)

In [4]:
N = 100
num_rw = 1000
m = 10

energy_rate, starting_M, starting_M_reverse = start_sequences(N_1, N_2, lmbd, p_match, p_cat, l, beta, x, y, N, num_rw, m)

print("---------------------------------------")
print("The starting M: ", starting_M)
print("-------------------------------------")

---------------------------------------
The starting M:  [[11.  0.  0.  0.  0.  0. 34.  7.  0. 16.  0.  0.  0. 21.  8.  0.  0.  0.
   0.  0.  0.  3.  0. 27.  0. 25.  0.  2. 17.  0. 14.  0. 24.  0. 44.  0.
   0.  0.  0. 28.  0. 30.  0.  0. 32.]
 [ 0.  6. 36. 11.  0.  0.  0. 30.  0.  0.  0.  0.  0.  0. 18.  0.  0.  0.
   0.  0.  0.  5.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 35.  0.
   0. 41.  0.  0.  0.  0.  0. 10.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 17. 34.  0. 18.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  8.  0. 28.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 29.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 28. 24. 20. 34.  0.  0.  0.  9.  3. 23.  0.  0.  0. 40.  4.
   0.  0.  0. 35. 26.  0.  0.  0.  7.  0. 17.  0. 18. 12.  0. 44.  0. 13.
  14.  0.  0. 42.  0. 19.  0.  0. 39.]
 [ 0.  0

In [5]:
#Checking the random walk R value from the Gelman-Rubin diagnostic
# from RandomWalkSampler import *
# n = int(1e3)
# traces_rw = []
# m = 10
# for i in np.arange(0,m):
#     #Run the sampler in this case
#     trace, energy, hamming, num_acc, runtime = rw_sampler(2*n, N_1, N_2, lmbd, p_match, M_truth, starting_M[i,:], starting_M_reverse[i,:], p_cat, l, beta, x, y, print_rate=10)
#     traces_rw.append(energy)
    
# #Testing the function to make the code scaleable
# R_energy_rw = gelman_rubin(traces_rw, n, m)
# print("-------------------------------------------------------------------")
# print("The scale reduction for the energy of the target_density: ",R_energy_rw)
# print("-------------------------------------------------------------------")

In [6]:
#Checking the Tabu sampler R value from the Gelman-Rubin diagnostic
# n = int(1e3)
# print_rate = 10
# T = 15
# thin_rate = 0.015
# traces_t = []
# m = 10
# for i in np.arange(0,m):
#     #Run the sampler in this case
#     trace, energy, hamming, alpha, num_iter, runtime = tabu_sampler(N_1, N_2, num_gens, starting_M[i,:], starting_M_reverse[i,:], g,
#                                                                     2*T, M_truth, thin_rate, print_rate, lmbd, p_match, l, p_cat, beta, x, y)
#     traces_t.append(energy)
    
# #Testing the function to make the code scaleable
# R_energy_t = gelman_rubin(traces_t, n, m)
# print("-------------------------------------------------------------------")
# print("The scale reduction for the energy of the target_density: ",R_energy_t)
# print("-------------------------------------------------------------------")

In [7]:
#Running the Gelman-Rubin diagnostic for all four samplers for N=100
n = int(1e2)
print_rate = 10
T_z = 1.5
T_t = 3.00
thin_rate_t = 0.03
thin_rate_z = 0.015
traces_rw = []
traces_pw = []
traces_z = []
traces_t = []
m = 10
for i in np.arange(0,m):
    #Run the random walk sampler
    trace_rw, energy_rw, hamming_rw, num_acc_rw, runtime_rw = rw_sampler(2*n, N_1, N_2, lmbd, p_match, M_truth, starting_M[i,:], starting_M_reverse[i,:], p_cat, l, beta, x, y, print_rate=10)
    #Run the pointwise sampler
    trace1_pw, energy_pw, hamming_pw, num_iter_pw, runtime_pw = pointwise_sampler(2*n, N_1, N_2, lmbd, p_match, g, M_truth, starting_M[i,:],
                                                                              starting_M_reverse[i,:], p_cat, l, beta, x, y, print_rate)
    
    #Run the Zanella sampler
    trace1_z, energy_z, hamming_z, num_iter_z, runtime_z = zanella_sampler(N_1, N_2,
                                                                                num_gens,
                                                                                starting_M[i,:], starting_M_reverse[i,:],
                                                                                g, 2*T_z, M_truth,
                                                                                thin_rate_z, print_rate, lmbd, p_match,
                                                                                l, p_cat, beta, x, y)
    
    #Run the Tabu sampler
    trace_t, energy_t, hamming_t, alpha_t, num_iter_t, runtime_t = tabu_sampler(N_1, N_2, num_gens, starting_M[i,:], starting_M_reverse[i,:], g,
                                                                    2*T_t, M_truth, thin_rate_t, print_rate, lmbd, p_match, l, p_cat, beta, x, y)
    traces_rw.append(energy_rw)
    traces_pw.append(energy_pw)
    traces_z.append(energy_z)
    traces_t.append(energy_t)
    
#Testing the function to make the code scaleable
R_energy_rw = gelman_rubin(traces_rw, n, m)
R_energy_pw = gelman_rubin(traces_pw, n, m)
R_energy_z = gelman_rubin(traces_z, n, m)
R_energy_t = gelman_rubin(traces_t, n, m)
print("-------------------------------------------------------------------")
print("The scale reduction for the energy of the target_density (random walk sampler): ",R_energy_rw)
print("-------------------------------------------------------------------")
print("-------------------------------------------------------------------")
print("The scale reduction for the energy of the target_density (Pointwise sampler): ",R_energy_pw)
print("-------------------------------------------------------------------")
print("-------------------------------------------------------------------")
print("The scale reduction for the energy of the target_density (Zanella sampler): ",R_energy_z)
print("-------------------------------------------------------------------")
print("-------------------------------------------------------------------")
print("The scale reduction for the energy of the target_density (Tabu sampler): ",R_energy_t)
print("-------------------------------------------------------------------")

Percent: [--------------------------------------->] 100%
Acceptance ratio:  0.32
Runtime:  0.62
Percent: [--------------------------------------->] 100%
Acceptance ratio:  0.84
Runtime:  9.85
Percent: [---------------------------------------->] 103%Runtime:  4.83
Percent: [--------------------------------------> ] 98%Average excursion length:  27.0
Runtime:  5.16
Percent: [--------------------------------------->] 100%
Acceptance ratio:  0.31
Runtime:  0.57
Percent: [--------------------------------------->] 100%
Acceptance ratio:  0.81
Runtime:  10.49
Percent: [--------------------------------------> ] 98%Runtime:  6.42
Percent: [------------------------------------>   ] 92%Average excursion length:  25.0
Runtime:  4.68
Percent: [--------------------------------------->] 100%
Acceptance ratio:  0.275
Runtime:  0.53
Percent: [--------------------------------------->] 100%
Acceptance ratio:  0.83
Runtime:  10.85
Percent: [--------------------------------------->] 99%Runtime:  4.6
Percen

In [8]:
#Running the Gelman-Rubin diagnostic for all four samplers for N=500
n = int(5e2)
print_rate = 10
T_z = 7.5
T_t = 15.0
thin_rate_t = 0.03
thin_rate_z = 0.015
traces_rw = []
traces_pw = []
traces_z = []
traces_t = []
m = 10
for i in np.arange(0,m):
    #Run the random walk sampler
    trace_rw, energy_rw, hamming_rw, num_acc_rw, runtime_rw = rw_sampler(2*n, N_1, N_2, lmbd, p_match, M_truth, starting_M[i,:], starting_M_reverse[i,:], p_cat, l, beta, x, y, print_rate=10)
    #Run the pointwise sampler
    trace1_pw, energy_pw, hamming_pw, num_iter_pw, runtime_pw = pointwise_sampler(2*n, N_1, N_2, lmbd, p_match, g, M_truth, starting_M[i,:],
                                                                              starting_M_reverse[i,:], p_cat, l, beta, x, y, print_rate)
    
    #Run the Zanella sampler
    trace1_z, energy_z, hamming_z, num_iter_z, runtime_z = zanella_sampler(N_1, N_2,
                                                                                num_gens,
                                                                                starting_M[i,:], starting_M_reverse[i,:],
                                                                                g, 2*T_z, M_truth,
                                                                                thin_rate_z, print_rate, lmbd, p_match,
                                                                                l, p_cat, beta, x, y)
    
    #Run the Tabu sampler
    trace_t, energy_t, hamming_t, alpha_t, num_iter_t, runtime_t = tabu_sampler(N_1, N_2, num_gens, starting_M[i,:], starting_M_reverse[i,:], g,
                                                                    2*T_t, M_truth, thin_rate_t, print_rate, lmbd, p_match, l, p_cat, beta, x, y)
    traces_rw.append(energy_rw)
    traces_pw.append(energy_pw)
    traces_z.append(energy_z)
    traces_t.append(energy_t)
    
#Testing the function to make the code scaleable
R_energy_rw = gelman_rubin(traces_rw, n, m)
R_energy_pw = gelman_rubin(traces_pw, n, m)
R_energy_z = gelman_rubin(traces_z, n, m)
R_energy_t = gelman_rubin(traces_t, n, m)
print("-------------------------------------------------------------------")
print("The scale reduction for the energy of the target_density (random walk sampler): ",R_energy_rw)
print("-------------------------------------------------------------------")
print("-------------------------------------------------------------------")
print("The scale reduction for the energy of the target_density (Pointwise sampler): ",R_energy_pw)
print("-------------------------------------------------------------------")
print("-------------------------------------------------------------------")
print("The scale reduction for the energy of the target_density (Zanella sampler): ",R_energy_z)
print("-------------------------------------------------------------------")
print("-------------------------------------------------------------------")
print("The scale reduction for the energy of the target_density (Tabu sampler): ",R_energy_t)
print("-------------------------------------------------------------------")

Percent: [--------------------------------------->] 100%
Acceptance ratio:  0.04
Runtime:  4.67
Percent: [--------------------------------------->] 100%
Acceptance ratio:  0.738
Runtime:  57.17
Percent: [--------------------------------------->] 101%Runtime:  17.54
Percent: [--------------------------------------> ] 99%Average excursion length:  13.0
Runtime:  19.63
Percent: [--------------------------------------->] 100%
Acceptance ratio:  0.021
Runtime:  4.71
Percent: [--------------------------------------->] 100%
Acceptance ratio:  0.736
Runtime:  54.16
Percent: [--------------------------------------> ] 98%Runtime:  24.16
Percent: [--------------------------------------->] 100%Average excursion length:  13.0
Runtime:  25.75
Percent: [--------------------------------------->] 100%
Acceptance ratio:  0.022
Runtime:  4.62
Percent: [--------------------------------------->] 100%
Acceptance ratio:  0.751
Runtime:  56.89
Percent: [------------------------------------->  ] 96%Runtime:  2

In [9]:
#Running the Gelman-Rubin diagnostic for all four samplers for N=1000
n = int(1e3)
print_rate = 10
T_z = 15
T_t = 30
thin_rate_t = 0.03
thin_rate_z = 0.015
traces_rw = []
traces_pw = []
traces_z = []
traces_t = []
m = 10
for i in np.arange(0,m):
    #Run the random walk sampler
    trace_rw, energy_rw, hamming_rw, num_acc_rw, runtime_rw = rw_sampler(2*n, N_1, N_2, lmbd, p_match, M_truth, starting_M[i,:], starting_M_reverse[i,:], p_cat, l, beta, x, y, print_rate=10)
    #Run the pointwise sampler
    trace1_pw, energy_pw, hamming_pw, num_iter_pw, runtime_pw = pointwise_sampler(2*n, N_1, N_2, lmbd, p_match, g, M_truth, starting_M[i,:],
                                                                              starting_M_reverse[i,:], p_cat, l, beta, x, y, print_rate)
    
    #Run the Zanella sampler
    trace1_z, energy_z, hamming_z, num_iter_z, runtime_z = zanella_sampler(N_1, N_2,
                                                                                num_gens,
                                                                                starting_M[i,:], starting_M_reverse[i,:],
                                                                                g, 2*T_z, M_truth,
                                                                                thin_rate_z, print_rate, lmbd, p_match,
                                                                                l, p_cat, beta, x, y)
    
    #Run the Tabu sampler
    trace_t, energy_t, hamming_t, alpha_t, num_iter_t, runtime_t = tabu_sampler(N_1, N_2, num_gens, starting_M[i,:], starting_M_reverse[i,:], g,
                                                                    2*T_t, M_truth, thin_rate_t, print_rate, lmbd, p_match, l, p_cat, beta, x, y)
    traces_rw.append(energy_rw)
    traces_pw.append(energy_pw)
    traces_z.append(energy_z)
    traces_t.append(energy_t)
    
#Testing the function to make the code scaleable
R_energy_rw = gelman_rubin(traces_rw, n, m)
R_energy_pw = gelman_rubin(traces_pw, n, m)
R_energy_z = gelman_rubin(traces_z, n, m)
R_energy_t = gelman_rubin(traces_t, n, m)
print("-------------------------------------------------------------------")
print("The scale reduction for the energy of the target_density (random walk sampler): ",R_energy_rw)
print("-------------------------------------------------------------------")
print("-------------------------------------------------------------------")
print("The scale reduction for the energy of the target_density (Pointwise sampler): ",R_energy_pw)
print("-------------------------------------------------------------------")
print("-------------------------------------------------------------------")
print("The scale reduction for the energy of the target_density (Zanella sampler): ",R_energy_z)
print("-------------------------------------------------------------------")
print("-------------------------------------------------------------------")
print("The scale reduction for the energy of the target_density (Tabu sampler): ",R_energy_t)
print("-------------------------------------------------------------------")

Percent: [--------------------------------------->] 100%
Acceptance ratio:  0.026
Runtime:  8.58
Percent: [--------------------------------------->] 100%
Acceptance ratio:  0.756
Runtime:  109.53
Percent: [--------------------------------------->] 100%Runtime:  37.07
Percent: [--------------------------------------->] 99%Average excursion length:  13.0
Runtime:  43.73
Percent: [--------------------------------------->] 100%
Acceptance ratio:  0.0305
Runtime:  8.86
Percent: [--------------------------------------->] 100%
Acceptance ratio:  0.7325
Runtime:  109.12
Percent: [--------------------------------------> ] 99%Runtime:  46.91
Percent: [--------------------------------------->] 100%Average excursion length:  14.0
Runtime:  51.3
Percent: [--------------------------------------->] 100%
Acceptance ratio:  0.0155
Runtime:  9.04
Percent: [--------------------------------------->] 100%
Acceptance ratio:  0.724
Runtime:  110.93
Percent: [--------------------------------------->] 100%Runt